In [1]:
#general packages
import pandas as pd
import numpy as np
import tifffile as tf
from tqdm import tqdm
#import custom function
from colocalization import colocalizing_dots

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_image(img_2d, df1, df2, zmax, add_trace = True):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=600,
        height=600,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax,
        #animation_frame=0
    )
    if add_trace==True:
        fig.add_trace(go.Scattergl(
            x=df1.x,
            y=df1.y,
            mode='markers',
            marker_symbol='cross',
            marker=dict(
                #maxdisplayed=1000,
                size=4
                ),
            name = "first dot"
            ))
        fig.add_trace(go.Scattergl(
            x=df2.x,
            y=df2.y,
            mode='markers',
            marker_symbol='cross',
            marker=dict(
                #maxdisplayed=1000,
                size=4
                ),
            name = "second dot"
            ))
    fig.show()

In [ ]:
#read in dots file
pos = 0
dot_src = f"/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/dots_detected/Pos{pos}/locations_z_0.csv"
dots_df = pd.read_csv(dot_src, index_col = 0)

In [ ]:
#isolate channel and hyb you would like to compare
dots_df1 = dots_df[(dots_df["hyb"]==0)&(dots_df["ch"]==1)]
dots_df2 = dots_df[(dots_df["hyb"]==12)&(dots_df["ch"]==1)]

eff, df = colocalizing_dots(dots_df1, dots_df2, radius=2, return_dots_not_coloc=False)

In [ ]:
#read image for overlay
hyb = 0
pos = 0
img_raw = f'/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/aberration_corrected/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif'
img = tf.imread(img_raw)

In [ ]:
#look at shape
img.shape

In [ ]:
#plot to see if the dots make sense
plot_2d_image(img[0], df[0], df[1], zmax=3000, add_trace = True)

In [ ]:
#read image for overlay
hyb = 12
pos = 0
img_raw = f'/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/aberration_corrected/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif'
img = tf.imread(img_raw)

In [ ]:
#plot to see if the dots make sense
plot_2d_image(img[0], df[0], df[1], zmax=3000, add_trace = True)

# Check efficiency for other channels and other pos

In [ ]:
#specifiy hybs
hybs = [0,12]
#specify channel comparions
channels = [4,4]
#list for efficiency
eff_list = []
for pos in tqdm(np.arange(0,15,1)):
    #get specific position
    dot_src = f"/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/dots_detected/Pos{pos}/locations_z_0.csv"
    #read in csv
    try:
        dots_df = pd.read_csv(dot_src, index_col = 0)
    except FileNotFoundError:
        continue
    #isolate channel and hyb you would like to compare
    dots_df1 = dots_df[(dots_df["hyb"]==hybs[0])&(dots_df["ch"]==channels[0])]
    dots_df2 = dots_df[(dots_df["hyb"]==hybs[1])&(dots_df["ch"]==channels[1])]
    #get efficiency
    eff, _ = colocalizing_dots(dots_df1, dots_df2, radius=3.0)
    eff_list.append([pos,eff])

In [ ]:
np.mean(np.array(eff_list)[:,1])

In [ ]:
#check pos 6
pos = 4
dot_src = f"/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/dots_detected/Pos{pos}/locations_z_0.csv"
dots_df = pd.read_csv(dot_src, index_col = 0)

In [ ]:
#isolate channel and hyb you would like to compare
dots_df1 = dots_df[(dots_df["hyb"]==0)&(dots_df["ch"]==1)]
dots_df2 = dots_df[(dots_df["hyb"]==12)&(dots_df["ch"]==1)]

eff, df = colocalizing_dots(dots_df1, dots_df2, radius=2)

In [ ]:
#read image for overlay
hyb = 12
pos = 4
img_raw = f'/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/aberration_corrected/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif'
img = tf.imread(img_raw)

In [ ]:
#plot to see if the dots make sense
plot_2d_image(img[0],df[0], df[1], zmax=3000, add_trace = True)

# Dot displacement analysis

In [ ]:
from localization_error import dot_displacement

In [ ]:
#read in locations
dots_df = pd.read_csv("/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/dots_detected/Pos0/locations_z_0.csv")

In [ ]:
#isolate channel and hyb you would like to compare
dots_df1 = dots_df[(dots_df["hyb"]==0)&(dots_df["ch"]==1)]
dots_df2 = dots_df[(dots_df["hyb"]==12)&(dots_df["ch"]==1)]

eff, df = colocalizing_dots(dots_df1, dots_df2, radius=2, return_dots_not_coloc=False)

In [ ]:
displacement = dot_displacement(dots_df1, dots_df2, radius=2)

In [ ]:
displacement 

# Colocalization output comparing 1st and last hyb

In [3]:
import pandas as pd

In [4]:
#read colocalization output and combine
comb_df = []
for ch in [1,2,3]:
    df = pd.read_csv(f"/groups/CaiLab/personal/Lex/raw/092222_150_nih_3t3/notebook_pyfiles/hyb_colocalization/hyb_coloc_channel_{ch}.csv", index_col=0)
    comb_df.append(df)
comb_df = pd.concat(comb_df).reset_index(drop=True)

In [5]:
#calculate mean for each channel grouped by radii
mean_and_sem = []
for ch in [1,2,3]:
    mean = comb_df[comb_df["ch"]==ch].groupby("radii").mean().reset_index()[["radii","ch","eff"]]
    mean.columns = ["radii","ch","mean eff"]
    sem = comb_df[comb_df["ch"]==ch].groupby("radii").sem().reset_index()["eff"]
    mean["sem eff"] = sem.values
    mean_and_sem.append(mean)
final = pd.concat(mean_and_sem).reset_index(drop=True)

In [6]:
final

,radii,ch,mean eff,sem eff
0,0.75,1,0.87876,0.024918
1,1.00,1,0.91124,0.017821
2,2.00,1,0.93772,0.004220
3,0.75,2,0.83500,0.016403
4,1.00,2,0.88020,0.008256
5,2.00,2,0.90032,0.006127
6,0.75,3,0.85288,0.019205
7,1.00,3,0.91104,0.008649
8,2.00,3,0.93788,0.004421
